In [ ]:
#Configurazione e Bootstrap

import os
import sys
import shutil

print(">>> Executing Environment Reset...")
try:
    os.chdir('/content/')

    #Usiamo shutil per una rimozione più sicura in ambiente Python
    if os.path.exists('Hawk-AI-CV-Project'):
        shutil.rmtree('Hawk-AI-CV-Project')
except Exception as e:
    print(f"Reset non necessario o errore minore: {e}")

print(">>> Cloning Repository...")
!git clone https://github.com/leonardoCosta02/Hawk-AI-CV-Project.git

#Path configuration
repo_name = "Hawk-AI-CV-Project"
%cd {repo_name}

#Root del progetto nel path per gli import assoluti
if os.getcwd() not in sys.path:
    sys.path.insert(0, os.getcwd())

print(">>> Installing Runtime Dependencies...")

#Aggiornamento per evitare conflitti di versione tra numpy e opencv
!pip install --upgrade numpy opencv-python matplotlib pandas

#Setup check
print("\n>>> Setup Complete. Active Runtime Root:")
print(os.getcwd())

try:
    import numpy as np
    import cv2 as cv
    import pandas as pd
    print(f">>> Dependencies verified: OpenCV {cv.__version__} | Numpy {np.__version__}")

    #Check se i file necessari sono presenti
    file_necessari = ['src/config.py', 'src/homography_calculator.py']
    for f in file_necessari:
        if os.path.exists(f):
            print(f"File di sistema trovato: {f}")
        else:
            print(f"ATTENZIONE: File mancante: {f}")

except ImportError as e:
    print(f"CRITICAL ERROR: Dependency installation failed - {e}")


In [8]:
#Configurazione del giudice
import cv2 as cv
import numpy as np

#LIMITI CAMPO

COURT_DIMENSIONS = {
    'X_MIN': -2.50,   # Margine laterale sinistro (corridoi)
    'X_MAX': 11.0,    # Margine laterale destro (corridoi)

    'Y_MIN': -7.0,    # Fondo campo (Giocatore in basso)

    'Y_MAX': 22.0
}


def is_point_in(x, y):
    if x is None or y is None: return False
    return (COURT_DIMENSIONS['X_MIN'] <= x <= COURT_DIMENSIONS['X_MAX'] and
            COURT_DIMENSIONS['Y_MIN'] <= y <= COURT_DIMENSIONS['Y_MAX'])

class HawkEyeJudge:
    def __init__(self, homography_matrix):
        self.H = np.array(homography_matrix, dtype=np.float32)

    def pixel_to_world(self, u, v):
        src_pt = np.array([[[u, v]]], dtype=np.float32)
        dst_pt = cv.perspectiveTransform(src_pt, self.H)
        return dst_pt[0][0][0], dst_pt[0][0][1]

    def detect_bounces(self, trajectory):
        bounces = []
        if len(trajectory) < 5: return bounces

        last_bounce_frame = -50

        for t in range(2, len(trajectory) - 2):
            prev, curr, nxt = trajectory[t-1], trajectory[t], trajectory[t+1]
            if prev is None or curr is None or nxt is None: continue

            v_vel_in  = curr[1] - prev[1]
            v_vel_out = nxt[1] - curr[1]

            #Soglia Velocità
            if v_vel_in > 1.5 and v_vel_out <= 0:
                #Filtro Temporale
                if (t - last_bounce_frame) > 15:

                    wx, wy = self.pixel_to_world(prev[0], prev[1])

                    if -10.0 <= wy <= 20.0 and -5.0 <= wx <= 12.0:
                        bounces.append({'frame': t, 'pixel': prev})
                        last_bounce_frame = t
                        print(f"Rimbalzo RILEVATO Frame {t}: Y={wy:.2f}m")

        return bounces

    def adjudicate_video(self, video_path, trajectory, output_path):
        cap = cv.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Errore apertura video: {video_path}")
            return

        fps = cap.get(cv.CAP_PROP_FPS)
        w, h = int(cap.get(cv.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
        out = cv.VideoWriter(output_path, cv.VideoWriter_fourcc(*'mp4v'), fps, (w, h))

        bounces = self.detect_bounces(trajectory)
        print(f"TOTALE RIMBALZI VALIDI: {len(bounces)}")

        decisions = {}
        for b in bounces:
            f_idx = b['frame']
            px, py = b['pixel']
            wx, wy = self.pixel_to_world(px, py)

            if wx is not None:
                verdict = "IN" if is_point_in(wx, wy) else "OUT"
                color = (0, 255, 0) if verdict == "IN" else (0, 0, 255)

                decisions[f_idx] = {
                    'verdict': verdict,
                    'coords': (wx, wy),
                    'color': color
                }
                print(f"   -> Frame {f_idx} | World: X={wx:.2f}, Y={wy:.2f} | {verdict}")

        frame_count = 0
        display_timer = 0
        current_display = None

        while True:
            ret, frame = cap.read()
            if not ret: break

            if frame_count in decisions:
                current_display = decisions[frame_count]

                #Velocità display
                display_timer = int(fps * 0.8)

            if frame_count < len(trajectory) and trajectory[frame_count] is not None:
                cv.circle(frame, (int(trajectory[frame_count][0]), int(trajectory[frame_count][1])), 4, (0, 255, 255), -1)

            if display_timer > 0 and current_display:
                cv.rectangle(frame, (40, 40), (450, 160), (0,0,0), -1)
                cv.putText(frame, current_display['verdict'], (60, 110), cv.FONT_HERSHEY_SIMPLEX, 2, current_display['color'], 5)
                cv.putText(frame, f"X:{current_display['coords'][0]:.2f} Y:{current_display['coords'][1]:.2f}", (60, 145), cv.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)
                display_timer -= 1

            out.write(frame)
            frame_count += 1

        cap.release()
        out.release()
        print(f"Video salvato in: {output_path}")

In [ ]:
#Esecuzione un video per volta+ autoscale
import pandas as pd
import numpy as np
import glob
import os
import cv2 as cv
from google.colab import files


NUMERO_VIDEO = 4 #Numero video


#Matrice H (in questo caso versione 'Cemento')
H_MATRIX = np.array([[ 6.50980565e-02,  2.65116930e-02, -3.04981716e+01],
 [ 3.86372362e-04, -1.97434738e-01,  7.91002378e+01],
 [ 1.76143090e-04,  6.49680276e-03,  1.00000000e+00]])

#Risoluzione di Riferimento della matrice H
CALIBRATION_W, CALIBRATION_H = 1075, 613

print("Inizializzazione Hawk-Eye Judge...")
judge = HawkEyeJudge(H_MATRIX)

#Nomi dei file in base al numero scelto
nome_csv = f"DATI_video{NUMERO_VIDEO}.csv"
nome_base = f"video{NUMERO_VIDEO}"

#Controlla se il CSV esiste
if not os.path.exists(nome_csv):
    print(f"ERRORE: Il file '{nome_csv}' non è stato trovato")
    print(f"   Assicurati di aver generato il CSV con YOLO e di averlo caricato.")
else:
    print(f"\n" + "="*50)
    print(f"File Dati trovato: {nome_csv}")

    #Cerca il video corrispondente (.mov, .mp4, .avi)
    video_path = None
    possible_extensions = [".mov", ".mp4", ".avi"]

    #Cerca nella cartella corrente o nella sottocartella del progetto
    possibili_percorsi = [f"{nome_base}{ext}" for ext in possible_extensions] + \
                         [f"/content/Hawk-AI-CV-Project/{nome_base}{ext}" for ext in possible_extensions]

    for p in possibili_percorsi:
        if os.path.exists(p):
            video_path = p
            break

    if video_path:
        #Vediamo risoluzione reale del video
        cap_temp = cv.VideoCapture(video_path)
        if not cap_temp.isOpened():
             print(f"Impossibile aprire il file video: {video_path}")
        else:
            real_w = int(cap_temp.get(cv.CAP_PROP_FRAME_WIDTH))
            real_h = int(cap_temp.get(cv.CAP_PROP_FRAME_HEIGHT))
            cap_temp.release()

            print(f"Analisi Video: {video_path}")
            print(f"Risoluzione rilevata: {real_w}x{real_h}")

            # Calcolo automatico della scala
            # Se il video è 640x360, scalerà le coordinate per arrivare ad es a 1075x613
            scale_u = CALIBRATION_W / real_w
            scale_v = CALIBRATION_H / real_h
            print(f"Fattore di scala applicato: X: {scale_u:.3f} | Y: {scale_v:.3f}")

            #Carichiamo i dati
            df = pd.read_csv(nome_csv)
            #Creiamo una lista vuota lunga quanto il video
            max_frame = int(df['frame'].max())
            trajectory_list = [None] * (max_frame + 200)

            for _, row in df.iterrows():
                f = int(row['frame'])
                #Applichiamo la scala letta dal video
                u = float(row['u']) * scale_u
                v = float(row['v']) * scale_v

                if not (np.isnan(u) or np.isnan(v)):
                    trajectory_list[f] = (u, v)

            #Giudizio finale nel video
            output_name = f"FINAL_JUDGE_{nome_base}.mp4"
            judge.adjudicate_video(video_path, trajectory_list, output_name)

            if os.path.exists(output_name):
                print(f"💾 Download automatico in corso...")
                files.download(output_name)
    else:
        print(f"Video '{nome_base}' non trovato (cercato .mp4, .mov, .avi).")
        print("Carica il file video nella barra a sinistra")